## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "BMW Championship"
tournament_date = "8/17/2025"  # Ending date of tournament
old_course = "Caves Valley Golf Club"
tournament_id = "R2025028"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Procore Championship"
new_ending_date = "9/14/2025"
new_course = "Silverado Resort and Spa North"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025028 (BMW Championship), year: 20250
✅ 49 new rows added for 'BMW Championship'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-08-17,R2025028,BMW Championship,Caves Valley Golf Club,Scottie Scheffler,1,1,-4,-5,-3,-3,"$3,600,000.00","2,000.000"
1,2025,2025-08-17,R2025028,BMW Championship,Caves Valley Golf Club,Robert MacIntyre,2,2,-8,-6,-2,+3,"$2,160,000.00","1,200.000"
2,2025,2025-08-17,R2025028,BMW Championship,Caves Valley Golf Club,Maverick McNealy,3,3,E,-6,-1,-4,"$1,360,000.00",760.000
3,2025,2025-08-17,R2025028,BMW Championship,Caves Valley Golf Club,Tommy Fleetwood,T4,4,-5,-1,-1,-3,"$910,000.00",490.000
4,2025,2025-08-17,R2025028,BMW Championship,Caves Valley Golf Club,Sam Burns,T4,4,-2,-3,-3,-2,"$910,000.00",490.000


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,655.0,.1910,2025
1,Aaron Baddeley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572.0,.2283,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361.0,.4119,2025
3,Aaron Rai,11.0,0.942,21.0,0.391,22.0,0.476,66.0,0.077,120.0,...,289.6,2.0,73.85%,11.0,70.52%,92.0,59.16%,34.0,2.6079,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,960.0,.0958,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

✅ Cleaned and updated 207 rows in 'odds' table.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG
5797,2024,The Open Championship,2024-07-21,Rory McIlroy,17/2,8.5,Open Championship,Rory McIlroy
5798,2024,The Open Championship,2024-07-21,Jon Rahm,20/1,20.0,Open Championship,Jon Rahm
5799,2024,The Open Championship,2024-07-21,Scottie Scheffler,11/2,5.5,Open Championship,Scottie Scheffler
5800,2024,The Open Championship,2024-07-21,Viktor Hovland,30/1,30.0,Open Championship,Viktor Hovland
5801,2024,The Open Championship,2024-07-21,Cameron Smith,50/1,50.0,Open Championship,Cameron Smith


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 10 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
956,2016,Silverado Resort and Spa North,464,Frys.com Open,2015-10-18
744,2017,Silverado Resort and Spa North,464,Safeway Open,2016-10-16
888,2017,Silverado Resort and Spa North,464,Frys.com Open,2015-10-18
600,2018,Silverado Resort and Spa North,464,Safeway Open,2017-10-08
456,2019,Silverado Resort and Spa North,464,Safeway Open,2018-10-07
312,2020,Silverado Resort and Spa North,464,Safeway Open,2019-09-29
156,2021,Silverado Resort and Spa North,464,Safeway Open,2020-09-13
0,2022,Silverado Resort and Spa North,464,Fortinet Championship,2021-09-19
1244,2023,Silverado Resort and Spa North,R2023543,Fortinet Championship,2023-09-17
1556,2024,Silverado Resort and Spa North,R2024464,Procore Championship,2024-09-15


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2015-10-18 — Frys.com Open (515 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,6,156.00,40.0,10.40,0,2015-10-18,Frys.com Open
1,Adam Bland,1,0,0.00,0.0,0.00,0,2015-10-18,Frys.com Open
2,Adam Hadwin,19,12,434.45,63.2,22.87,3,2015-10-18,Frys.com Open



📆 2016-10-16 — Safeway Open (518 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,10,370.0,62.5,23.12,3,2016-10-16,Safeway Open
1,Aaron Wise,2,1,0.0,50.0,0.00,1,2016-10-16,Safeway Open
2,Abraham Ancer,9,4,96.5,44.4,10.72,0,2016-10-16,Safeway Open



📆 2017-10-08 — Safeway Open (514 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,11,3,30.17,27.3,2.74,0,2021-09-19,Fortinet Championship
1,Aaron Pike,1,0,0.00,0.0,0.00,0,2021-09-19,Fortinet Championship
2,Aaron Rai,3,2,0.00,66.7,0.00,2,2021-09-19,Fortinet Championship



📆 2023-09-17 — Fortinet Championship (588 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,10,325.036,62.5,20.31,0,2023-09-17,Fortinet Championship
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-09-17,Fortinet Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-09-17,Fortinet Championship



📆 2024-09-15 — Procore Championship (535 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,9,151.825,64.3,10.84,0,2024-09-15,Procore Championship
1,Aaron Cockerill,2,0,0.000,0.0,0.00,0,2024-09-15,Procore Championship
2,Aaron Rai,21,18,1767.657,85.7,84.17,14,2024-09-15,Procore Championship


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2015-10-18 — Frys.com Open (515 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jordan Niebrugge,1,6.0,8.66,2015-10-18,Frys.com Open
1,Anthony Wall,1,12.0,17.31,2015-10-18,Frys.com Open
2,Ashley Chesters,1,12.0,17.31,2015-10-18,Frys.com Open



📆 2016-10-16 — Safeway Open (518 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jared du Toit,1,9.0,12.98,2016-10-16,Safeway Open
1,Matthew Southgate,1,12.0,17.31,2016-10-16,Safeway Open
2,Dustin Johnson,14,15.5,5.72,2016-10-16,Safeway Open



📆 2017-10-08 — Safeway Open (514 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Taylor Pendrith,1,11.0,15.87,2021-09-19,Fortinet Championship
1,Marcel Siem,1,15.0,21.64,2021-09-19,Fortinet Championship
2,Louis Oosthuizen,11,17.7,7.12,2021-09-19,Fortinet Championship



📆 2023-09-17 — Fortinet Championship (588 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,18,7.6,2.58,2023-09-17,Fortinet Championship
1,Grant Forrest,1,11.0,15.87,2023-09-17,Fortinet Championship
2,Henrik Stenson,1,13.0,18.76,2023-09-17,Fortinet Championship



📆 2024-09-15 — Procore Championship (535 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,18,6.4,2.17,2024-09-15,Procore Championship
1,Xander Schauffele,19,9.9,3.30,2024-09-15,Procore Championship
2,Kevin Roy,1,12.0,17.31,2024-09-15,Procore Championship


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Frys.com Open on 2015-10-18


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,31.0,44.72,2015-10-18,Silverado Resort and Spa North,Frys.com Open
1,Adam Hadwin,1,53.0,76.46,2015-10-18,Silverado Resort and Spa North,Frys.com Open
2,Alex Cejka,1,90.0,129.84,2015-10-18,Silverado Resort and Spa North,Frys.com Open



🏌️‍♂️ Course history for Safeway Open on 2016-10-16


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,2,43.0,39.14,2016-10-16,Silverado Resort and Spa North,Safeway Open
1,Aaron Wise,1,90.0,129.84,2016-10-16,Silverado Resort and Spa North,Safeway Open
2,Abraham Ancer,1,90.0,129.84,2016-10-16,Silverado Resort and Spa North,Safeway Open



🏌️‍♂️ Course history for Safeway Open on 2017-10-08


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,7,52.9,25.44,2021-09-19,Silverado Resort and Spa North,Fortinet Championship
1,Aaron Wise,4,90.0,55.92,2021-09-19,Silverado Resort and Spa North,Fortinet Championship
2,Abraham Ancer,3,72.3,52.15,2021-09-19,Silverado Resort and Spa North,Fortinet Championship



🏌️‍♂️ Course history for Fortinet Championship on 2023-09-17


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,7,50.9,24.48,2023-09-17,Silverado Resort and Spa North,Fortinet Championship
1,Aaron Rai,1,90.0,129.84,2023-09-17,Silverado Resort and Spa North,Fortinet Championship
2,Aaron Wise,3,90.0,64.92,2023-09-17,Silverado Resort and Spa North,Fortinet Championship



🏌️‍♂️ Course history for Procore Championship on 2024-09-15


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,7,54.1,26.02,2024-09-15,Silverado Resort and Spa North,Procore Championship
1,Aaron Rai,1,90.0,129.84,2024-09-15,Silverado Resort and Spa North,Procore Championship
2,Aaron Wise,2,90.0,81.92,2024-09-15,Silverado Resort and Spa North,Procore Championship


In [8]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

## Training Dataset

In [9]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Patton Kizzire,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Paul Casey,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.5,1382.35,81.31,8.0,37.7,13.04,NaN,NaN,1
2,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Scott Piercy,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.7,1357.27,79.84,4.0,52.3,18.09,90.0,129.84,1
3,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Cody Gribble,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,0.00,0.00,0.0,75.0,46.60,NaN,NaN,1
4,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Phil Mickelson,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,830.50,55.37,7.0,41.7,15.04,NaN,NaN,1
5,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Mackenzie Hughes,T13,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Jon Rahm,T15,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.00,0.00,2.0,34.5,31.40,NaN,NaN,1
7,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Vaughn Taylor,T15,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.3,181.00,30.17,3.0,42.3,21.74,NaN,NaN,1
8,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Bill Haas,T20,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,1179.93,73.75,4.0,44.0,15.53,NaN,NaN,1
9,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Keegan Bradley,T22,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.4,788.63,46.39,3.0,38.7,13.39,NaN,NaN,0


In [10]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
1468,2024,2024-09-15,Procore Championship,Silverado Resort and Spa North,Patton Kizzire,1,1,48.0,0.456,119.0,-0.080,13.0,0.498,94.0,0.037,120.0,-0.101,5.0,25.05%,99.0,3.05,5.0,3.96,70.0,4.56,39.0,142.0,78.0,302.7,64.0,62.44%,4.0,72.42%,59.0,61.51%,98.0,1.2673,200.0,25.0,129.450,10.79,0.0,72.3,28.19,74.0,38.03,1
1469,2024,2024-09-15,Procore Championship,Silverado Resort and Spa North,David Lipsky,2,2,104.0,-0.013,162.0,-0.415,52.0,0.263,59.0,0.139,158.0,-0.385,125.0,21.35%,126.0,3.07,141.0,4.05,101.0,4.59,165.0,249.0,163.0,290.6,86.0,60.61%,82.0,67.30%,154.0,55.71%,159.0,0.8612,400.0,29.4,113.792,6.69,0.0,75.9,26.26,47.3,34.12,1
1470,2024,2024-09-15,Procore Championship,Silverado Resort and Spa North,Patrick Fishburn,3,3,62.0,0.368,7.0,0.630,119.0,-0.092,136.0,-0.170,66.0,0.138,77.0,22.59%,52.0,3.02,43.0,3.99,9.0,4.48,7.0,105.0,22.0,311.0,83.0,60.79%,1.0,73.18%,44.0,62.14%,123.0,1.0818,60.0,15.4,123.417,9.49,0.0,78.5,29.75,NaN,NaN,1
1471,2024,2024-09-15,Procore Championship,Silverado Resort and Spa North,Ben Silverman,T4,4,82.0,0.141,80.0,0.094,109.0,-0.022,77.0,0.069,4.0,0.640,92.0,22.29%,21.0,3.00,10.0,3.97,90.0,4.58,136.0,204.0,147.0,294.1,57.0,63.16%,28.0,69.74%,11.0,65.01%,127.0,1.0593,125.0,68.8,313.517,19.59,0.0,51.6,18.21,59.0,53.70,1
1472,2024,2024-09-15,Procore Championship,Silverado Resort and Spa North,Mackenzie Hughes,T4,4,118.0,-0.074,150.0,-0.277,134.0,-0.215,6.0,0.418,2.0,0.688,85.0,22.41%,74.0,3.04,85.0,4.01,15.0,4.50,172.0,273.0,117.0,298.4,156.0,55.05%,162.0,63.09%,10.0,65.40%,61.0,1.9215,50.0,82.6,1069.366,46.49,7.0,42.5,13.37,58.0,32.37,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [11]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Patton Kizzire,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.0763,1000.0,NaN,0.00,NaN,NaN,90.0,NaN,NaN,NaN,1
1,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Paul Casey,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.0763,1000.0,76.5,1382.35,81.31,8.0,37.7,13.04,NaN,NaN,1
2,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Scott Piercy,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.0763,1000.0,64.7,1357.27,79.84,4.0,52.3,18.09,90.0,129.84,1
3,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Cody Gribble,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.0763,1000.0,50.0,0.00,0.00,0.0,75.0,46.60,NaN,NaN,1
4,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Phil Mickelson,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.0763,1000.0,80.0,830.50,55.37,7.0,41.7,15.04,NaN,NaN,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [12]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Patton Kizzire,2,2,108.35886,-0.047578,103.907391,-0.022097,104.722173,-0.015804,103.703473,-0.010009,100.965272,-0.00568,100.729297,20.887427,92.644444,3.059022,94.640569,4.034564,97.911111,4.646053,102.561888,197.576135,99.666073,295.89537,97.910062,60.427854,95.272242,66.249689,100.091719,58.741621,1000.0,0.0763,1000.0,50.878639,0.00,17.871593,1.425054,90.0,31.487024,59.073204,59.067134,1
1,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Paul Casey,T3,3,108.35886,-0.047578,103.907391,-0.022097,104.722173,-0.015804,103.703473,-0.010009,100.965272,-0.00568,100.729297,20.887427,92.644444,3.059022,94.640569,4.034564,97.911111,4.646053,102.561888,197.576135,99.666073,295.89537,97.910062,60.427854,95.272242,66.249689,100.091719,58.741621,1000.0,0.0763,1000.0,76.500000,1382.35,81.310000,8.000000,37.7,13.040000,59.073204,59.067134,1
2,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Scott Piercy,T3,3,108.35886,-0.047578,103.907391,-0.022097,104.722173,-0.015804,103.703473,-0.010009,100.965272,-0.00568,100.729297,20.887427,92.644444,3.059022,94.640569,4.034564,97.911111,4.646053,102.561888,197.576135,99.666073,295.89537,97.910062,60.427854,95.272242,66.249689,100.091719,58.741621,1000.0,0.0763,1000.0,64.700000,1357.27,79.840000,4.000000,52.3,18.090000,90.000000,129.840000,1
3,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Cody Gribble,T8,8,108.35886,-0.047578,103.907391,-0.022097,104.722173,-0.015804,103.703473,-0.010009,100.965272,-0.00568,100.729297,20.887427,92.644444,3.059022,94.640569,4.034564,97.911111,4.646053,102.561888,197.576135,99.666073,295.89537,97.910062,60.427854,95.272242,66.249689,100.091719,58.741621,1000.0,0.0763,1000.0,50.000000,0.00,0.000000,0.000000,75.0,46.600000,59.073204,59.067134,1
4,2017,2015-10-18,Frys.com Open,Silverado Resort and Spa North,Phil Mickelson,T8,8,108.35886,-0.047578,103.907391,-0.022097,104.722173,-0.015804,103.703473,-0.010009,100.965272,-0.00568,100.729297,20.887427,92.644444,3.059022,94.640569,4.034564,97.911111,4.646053,102.561888,197.576135,99.666073,295.89537,97.910062,60.427854,95.272242,66.249689,100.091719,58.741621,1000.0,0.0763,1000.0,80.000000,830.50,55.370000,7.000000,41.7,15.040000,59.073204,59.067134,1


In [13]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [14]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [15]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,14400
1,Justin Thomas,10200
2,Russell Henley,10000
3,Sam Burns,9700
4,Patrick Cantlay,9500
5,Ben Griffin,9400
6,Cameron Young,9300
7,J.J. Spaun,9200
8,Collin Morikawa,9100
9,Maverick McNealy,9000


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Procore Championship,Scottie Scheffler,9/4,2.25
2,2025,Procore Championship,Russell Henley,14/1,14.00
3,2025,Procore Championship,Justin Thomas,16/1,16.00
4,2025,Procore Championship,Patrick Cantlay,18/1,18.00
5,2025,Procore Championship,Sam Burns,18/1,18.00
6,2025,Procore Championship,Cameron Young,20/1,20.00
7,2025,Procore Championship,Collin Morikawa,25/1,25.00
8,2025,Procore Championship,Ben Griffin,25/1,25.00
9,2025,Procore Championship,Maverick McNealy,25/1,25.00
10,2025,Procore Championship,J.J. Spaun,25/1,25.00


## Cut Percentage and FedEx Points

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-09-14,Procore Championship
1,Aaron Baddeley,13,5,45.038,38.5,3.46,1,2025-09-14,Procore Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2025-09-14,Procore Championship
3,Aaron Rai,20,16,941.647,80.0,47.08,6,2025-09-14,Procore Championship
4,Aaron Wise,5,1,6.333,20.0,1.27,0,2025-09-14,Procore Championship
5,Adam Hadwin,21,13,223.988,61.9,10.67,0,2025-09-14,Procore Championship
6,Adam Schenk,19,6,252.075,31.6,13.27,0,2025-09-14,Procore Championship
7,Adam Scott,17,14,451.128,82.4,26.54,2,2025-09-14,Procore Championship
8,Adam Svensson,18,8,127.914,44.4,7.11,0,2025-09-14,Procore Championship
9,Adrian Otaegui,1,0,0.000,0.0,0.00,0,2025-09-14,Procore Championship


## Recent Form

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,18,6.4,2.17,2025-09-14,Procore Championship
1,Jason Scrivener,1,8.0,11.54,2025-09-14,Procore Championship
2,Bubba Watson,1,14.0,20.20,2025-09-14,Procore Championship
3,Jon Rahm,4,15.8,9.82,2025-09-14,Procore Championship
4,Rory McIlroy,14,16.4,6.06,2025-09-14,Procore Championship
5,Andy Sullivan,1,17.0,24.53,2025-09-14,Procore Championship
6,Robin Williams,1,17.0,24.53,2025-09-14,Procore Championship
7,Joel Girrbach,1,20.0,28.85,2025-09-14,Procore Championship
8,Tommy Fleetwood,18,20.7,7.03,2025-09-14,Procore Championship
9,Kensei Hirata,1,21.0,30.30,2025-09-14,Procore Championship


## Course History

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,7,48.4,23.28,2025-09-14,Silverado Resort and Spa North,Procore Championship
1,Aaron Rai,1,90.0,129.84,2025-09-14,Silverado Resort and Spa North,Procore Championship
2,Aaron Wise,1,90.0,129.84,2025-09-14,Silverado Resort and Spa North,Procore Championship
3,Abraham Ancer,1,90.0,129.84,2025-09-14,Silverado Resort and Spa North,Procore Championship
4,Adam Hadwin,3,60.7,43.79,2025-09-14,Silverado Resort and Spa North,Procore Championship
5,Adam Long,5,71.2,39.74,2025-09-14,Silverado Resort and Spa North,Procore Championship
6,Adam Schenk,6,60.2,30.94,2025-09-14,Silverado Resort and Spa North,Procore Championship
7,Adam Scott,1,17.0,24.53,2025-09-14,Silverado Resort and Spa North,Procore Championship
8,Adam Svensson,4,29.3,18.21,2025-09-14,Silverado Resort and Spa North,Procore Championship
9,Adrien Dumont de Chassart,1,90.0,129.84,2025-09-14,Silverado Resort and Spa North,Procore Championship


## Merged Dataframe

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Adam Hadwin,150.0,-0.663,122.0,-0.167,152.0,-0.451,96.0,-0.046,80.0,0.075,75.0,22.06%,53.0,3.04,125.0,4.05,115.0,4.63,86.0,166,103.0,301.1,63.0,61.31%,134.0,64.33%,151.0,54.85%,151.0,.9364,2025.0,250.0,61.9,223.988,10.67,0.0,64.0,20.70,60.7,43.79,7000
1,Adam Schenk,133.0,-0.274,93.0,0.038,128.0,-0.187,124.0,-0.125,124.0,-0.161,118.0,20.81%,53.0,3.04,149.0,4.08,80.0,4.60,144.0,208,58.0,306.5,150.0,53.79%,154.0,61.72%,123.0,57.52%,215.0,.6906,2025.0,500.0,31.6,252.075,13.27,0.0,70.8,23.63,60.2,30.94,6400
2,Adam Svensson,53.0,0.350,128.0,-0.207,44.0,0.283,17.0,0.273,156.0,-0.526,143.0,20.11%,53.0,3.04,108.0,4.04,55.0,4.57,96.0,172,145.0,295.6,27.0,64.56%,37.0,68.61%,69.0,60.11%,244.0,.6163,2025.0,250.0,44.4,127.914,7.11,0.0,69.1,23.47,29.3,18.21,6800
3,Akshay Bhatia,48.0,0.384,98.0,0.015,11.0,0.590,142.0,-0.222,33.0,0.254,12.0,24.21%,10.0,2.99,23.0,3.99,124.0,4.64,118.0,187,128.0,298.2,59.0,61.55%,51.0,67.70%,144.0,55.63%,36.0,2.5760,2025.0,35.0,77.3,1408.657,64.03,2.0,40.5,12.92,63.0,45.44,8700
4,Alex Smalley,19.0,0.788,17.0,0.446,65.0,0.146,30.0,0.198,35.0,0.249,14.0,24.18%,53.0,3.04,3.0,3.95,38.0,4.56,5.0,80,44.0,308.4,36.0,63.62%,31.0,68.95%,9.0,64.21%,132.0,1.0962,2025.0,80.0,57.9,488.756,25.72,0.0,50.9,16.99,74.3,53.60,7600


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [22]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [23]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [24]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [25]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Adam Hadwin,150.000000,-0.663000,122.000000,-0.167000,152.00000,-0.451000,96.000000,-0.04600,80.000000,0.075000,75.000000,22.060000,53.000000,3.040000,125.000000,4.050000,115.000000,4.630000,86.000000,166.000000,103.000000,301.100000,63.00000,61.310000,134.000000,64.330000,151.000000,54.850000,151.0,0.9364,2025.0,250.0,61.900000,223.988,10.670000,0.000000,64.0,20.70,60.700000,43.790000,7000
1,Adam Schenk,133.000000,-0.274000,93.000000,0.038000,128.00000,-0.187000,124.000000,-0.12500,124.000000,-0.161000,118.000000,20.810000,53.000000,3.040000,149.000000,4.080000,80.000000,4.600000,144.000000,208.000000,58.000000,306.500000,150.00000,53.790000,154.000000,61.720000,123.000000,57.520000,215.0,0.6906,2025.0,500.0,31.600000,252.075,13.270000,0.000000,70.8,23.63,60.200000,30.940000,6400
2,Adam Svensson,53.000000,0.350000,128.000000,-0.207000,44.00000,0.283000,17.000000,0.27300,156.000000,-0.526000,143.000000,20.110000,53.000000,3.040000,108.000000,4.040000,55.000000,4.570000,96.000000,172.000000,145.000000,295.600000,27.00000,64.560000,37.000000,68.610000,69.000000,60.110000,244.0,0.6163,2025.0,250.0,44.400000,127.914,7.110000,0.000000,69.1,23.47,29.300000,18.210000,6800
3,Akshay Bhatia,48.000000,0.384000,98.000000,0.015000,11.00000,0.590000,142.000000,-0.22200,33.000000,0.254000,12.000000,24.210000,10.000000,2.990000,23.000000,3.990000,124.000000,4.640000,118.000000,187.000000,128.000000,298.200000,59.00000,61.550000,51.000000,67.700000,144.000000,55.630000,36.0,2.5760,2025.0,35.0,77.300000,1408.657,64.030000,2.000000,40.5,12.92,63.000000,45.440000,8700
4,Alex Smalley,19.000000,0.788000,17.000000,0.446000,65.00000,0.146000,30.000000,0.19800,35.000000,0.249000,14.000000,24.180000,53.000000,3.040000,3.000000,3.950000,38.000000,4.560000,5.000000,80.000000,44.000000,308.400000,36.00000,63.620000,31.000000,68.950000,9.000000,64.210000,132.0,1.0962,2025.0,80.0,57.900000,488.756,25.720000,0.000000,50.9,16.99,74.300000,53.600000,7600
5,Anders Albertson,93.663265,0.019592,88.459184,0.001622,89.27551,0.021827,87.428571,-0.00401,85.734694,-0.002969,87.122449,21.755102,76.877551,3.054898,76.387755,4.019796,82.122449,4.599694,80.693878,162.061224,83.010204,303.422449,79.05102,60.376122,78.489796,66.613673,82.765306,59.547347,1000.0,20.6129,2025.0,1000.0,52.688889,0.000,23.604127,1.579365,90.0,24.03,56.854255,43.465319,6100
6,Andrew Putnam,84.000000,0.146000,158.000000,-0.680000,38.00000,0.319000,3.000000,0.50700,11.000000,0.464000,106.000000,21.200000,10.000000,2.990000,11.000000,3.970000,76.000000,4.590000,88.000000,168.000000,162.000000,282.600000,6.00000,70.310000,12.000000,70.460000,2.000000,67.710000,137.0,1.0334,2025.0,100.0,55.600000,401.033,22.280000,0.000000,52.8,17.93,48.600000,27.120000,7400
7,Austin Cook,93.663265,0.019592,88.459184,0.001622,89.27551,0.021827,87.428571,-0.00401,85.734694,-0.002969,87.122449,21.755102,76.877551,3.054898,76.387755,4.019796,82.122449,4.599694,80.693878,162.061224,83.010204,303.422449,79.05102,60.376122,78.489796,66.613673,82.765306,59.547347,647.0,0.1934,2025.0,1000.0,50.000000,8.400,4.200000,1.000000,65.5,59.62,64.800000,33.300000,6200
8,Austin Eckroat,106.000000,-0.012000,51.000000,0.223000,60.00000,0.184000,160.000000,-0.41900,123.000000,-0.154000,92.000000,21.670000,53.000000,3.040000,98.000000,4.030000,142.000000,4.670000,41.000000,136.000000,110.000000,300.200000,26.00000,65.070000,67.000000,66.980000,148.000000,55.050000,93.0,1.4963,2025.0,125.0,56.500000,370.656,16.120000,0.000000,61.3,19.29,77.000000,70.090000,7200
9,Beau Hossler,127.000000,-0

## Correlations
**Last 4 Years**

In [26]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [28]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.92699773 0.91642788 0.92099673 0.92405578 0.93120523]
Average ROC AUC score: 0.92


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [32]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,J.J. Spaun,9200,0.988574,32.9,46.600000,25.0,81.8,7.0,2293.798,9.000000,40.000000,7.00000,83.000000,73.000000,54.000000,40.000000,23.000000,67.000000,23.000000,67.000000,54.00000,43.000000,79.000000
1,Ben Griffin,9400,0.979447,37.6,90.000000,25.0,76.9,17.0,2397.580,21.000000,56.000000,28.00000,48.000000,22.000000,53.000000,29.000000,23.000000,22.000000,79.000000,69.000000,92.00000,40.000000,25.000000
2,Akshay Bhatia,8700,0.966809,40.5,63.000000,35.0,77.3,36.0,1408.657,48.000000,98.000000,11.00000,142.000000,33.000000,12.000000,10.000000,23.000000,124.000000,118.000000,128.000000,59.00000,51.000000,144.000000
3,Patrick Cantlay,9500,0.938040,30.7,28.500000,18.0,83.3,21.0,1661.202,14.000000,35.000000,9.00000,90.000000,54.000000,22.000000,114.000000,2.000000,115.000000,51.000000,63.000000,81.00000,18.000000,102.000000
4,Justin Thomas,10200,0.918370,28.3,4.500000,16.0,89.5,5.0,2477.240,25.000000,81.000000,31.00000,24.000000,17.000000,3.000000,74.000000,6.000000,80.000000,148.000000,74.000000,146.00000,102.000000,76.000000
5,Maverick McNealy,9000,0.881941,35.4,64.500000,25.0,82.6,18.0,2547.214,51.000000,52.000000,48.00000,122.000000,21.000000,57.000000,86.000000,41.000000,55.000000,65.000000,60.000000,95.00000,58.000000,121.000000
6,Collin Morikawa,9100,0.848032,30.1,10.000000,25.0,88.2,8.0,1655.491,2.000000,13.000000,3.00000,111.000000,141.000000,38.000000,2.000000,58.000000,115.000000,52.000000,140.000000,5.00000,33.000000,85.000000
7,Russell Henley,10000,0.794115,24.4,90.000000,14.0,88.2,4.0,2794.884,6.000000,96.000000,13.00000,6.000000,32.000000,35.000000,10.000000,6.000000,67.000000,88.000000,159.000000,9.00000,24.000000,11.000000
8,Davis Thompson,8800,0.682830,51.5,19.500000,45.0,65.2,75.0,614.617,20.000000,14.000000,75.00000,32.000000,145.000000,32.000000,98.000000,58.000000,38.000000,14.000000,78.000000,29.00000,36.000000,127.000000
9,Cameron Champ,7400,0.675231,31.3,64.300000,100.0,83.3,264.0,215.705,93.663265,88.459184,89.27551,87.428571,85.734694,87.122449,76.877551,76.387755,82.122449,80.693878,83.010204,79.05102,78.489796,82.765306


### Save to CSV

In [33]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
